# Xedocs user guide

This tutorial serves as a basic introduction to xedocs. Xedocs is the replacement of CMT (Correction Management System) and will be used moving forward to store, save and access correction data as well as other metadata needed for the XENONnT experiment moving forward. The main goal with xedocs was to have a system with the following requierments: We want versioned data that is flexible enough to adapt to future changes, to have insertion rules and to follow time dependace requirements. 

In [ ]:
import strax
import straxen
import xedocs as xd
import numpy as np
import rframe
import pymongo
import matplotlib.pyplot as plt

In [ ]:
straxen.print_versions('strax straxen rframe xedocs'.split())

With xedocs we can use schemas to get different corrections from multiple sources such as bodega (refered to as 'detector numbers'), the xedocs databse, or even your own! Schemas are python classes with certain properties and functions to deal with the backend of storing data as well as rules such as preventing the deletion and overwritting of existing data!
<br>
Lets look at some of the schemas that are available!

In [ ]:
xd.list_schemas()

### Accessing Data

The development database is a database in which everyone can enter their own corrections, it will also have all the values for each correction. To avoid confusion make sure to label the version as something that cannot possibly be a real correction, here I will use test* 

In [ ]:
xd_db_dev = xd.development_db()

The straxen database on the other had will have the real value of all correction and data can only be insterted into this database after other memebers of the colaboration have agreed that his value should be inserted.

In [ ]:
xd_db_stx = xd.straxen_db()

The data can be returned to the user in a varaiety of formats, for example in a pandas dataframe structure:

#### Dataframe format

In [ ]:
xd_db_stx.electron_lifetimes.find_df(version='ONLINE')

In [ ]:
elife = xd_db_stx.electron_lifetimes.find_df(version='ONLINE') # if you try to 

In [ ]:
elife

#### list format

In [ ]:
elife = xd_db_stx.electron_lifetimes.find(version='ONLINE') # if you try to 

In [ ]:
list(elife)[:3]

#### Dict format

In [ ]:
elife = xd_db_stx.electron_lifetimes.find_dicts(version='ONLINE') # if you try to 

In [ ]:
elife[:3]

#### "Docs" format

In [ ]:
elife = xd_db_stx.electron_lifetimes.find_docs(version='ONLINE') # if you try to 

In [ ]:
elife[:3]

#### You can just get one data point if that is all you want!

In [ ]:
elife = xd_db_stx.electron_lifetimes.find_one(version='ONLINE') # if you try to 

In [ ]:
elife

##### You can pass many different arguments into the search parameters to make your scope as limited or as wide as you might want it, you can also use _sort options if you want the data to be returned in a particualr order.

In [ ]:
rel_ext_eff = xd_db_stx.rel_extraction_effs.find_docs(version='v3')

In [ ]:
rel_ext_eff[:3]

In [ ]:
# lets sort by time and only get the ab partition
rel_ext_eff = xd_db_stx.rel_extraction_effs.find_docs(version='v3', partition='ab', __sort='time')

In [ ]:
rel_ext_eff[-3:]

In [ ]:
rel_ext_eff = xd_db_stx.rel_extraction_effs.find_docs(version='v3', 
                                                  run_id = '027434', 
                                                  partition = 'ab')

In [ ]:
rel_ext_eff

##### Data stored in xedocs have multiple indecies, and there is a requierment that all indecies are unique, therefore you can only save new data when it has indecies where the combination of all indecies is unique

We can also access the Bodega data through schemas using 'DetectorNumber'

In [ ]:
xd_db_stx.detector_numbers.find_one(field='g1')

In [ ]:
se_gain = xd_db_stx.detector_numbers.find(field = 'se_gain')

In [ ]:
list(se_gain)

### Saving data using xedocs

To save new data using xedocs we need to know what indecies that particular schema has. By defualt all schemas will require a version and a run_id or time for the indecies and a value to actually upload. However other shcemas can have additional indecies you need to spesify such as algorithm for machine learning related currections such as 'mlp', 'cnn' and 'gcn', format of the data, among others. As such we need to first know what are the indecies required and then we can upload the data!

For this set of correction we will save the data to the development db.

In [ ]:
run_id = '047493'

In [ ]:
# Get the ONLINE of elife for out run_id
elife_online = xd_db_stx.electron_lifetimes.find_docs(version = 'ONLINE', run_id = run_id)

In [ ]:
list(xd_db_dev.electron_lifetimes.find(run_id = run_id))

In [ ]:
elife_new = xd.schemas.ElectronLifetime(value=elife_online[0].value*0.90, version='test*', 
                                        run_id=run_id, datasource = 'development_db',
                                        comments='giving a 10% decrease to Electron lifetime')

In [ ]:
xd_db_dev.electron_lifetimes.insert(elife_new)

In [ ]:
list(xd_db_dev.electron_lifetimes.find(run_id = run_id))

In [ ]:
elife_new = xd.schemas.ElectronLifetime(value=elife_online[0].value*0.80, version='test*2', 
                                        run_id=run_id, datasource = 'development_db',
                                        comments='giving a 10% decrease to Electron lifetime')

In [ ]:
# there is another way to save this data
xd.insert_docs('electron_lifetimes', elife_new, 'development_db')

In [ ]:
list(xd_db_dev.electron_lifetimes.find(run_id = run_id))

#### Now to save data in our own database!

You will not have access to the config file bellow, however if you have mongoDB installed locally you can just ommit the the host, username and password information and just connect to your local mongo_db. You can also call the database whatever you want.

In [ ]:
import config

host = config.mongo_rdb_url
username = config.mongo_rdb_username
password = config.mongo_rdb_password

In [ ]:
db = pymongo.MongoClient(host = host,
                        username = username,
                        password = password)['correctionsSandbox']

In [ ]:
xd.schemas.ElectronLifetime.find(version = 'test*', datasource = db['electron_lifetimes'])

In [ ]:
new_elife = xd.schemas.ElectronLifetime(value = 123456, version = 'test*', run_id = run_id)

In [ ]:
new_elife.save(db['electron_lifetimes'])

In [ ]:
xd.schemas.ElectronLifetime.find(version = 'test*', datasource = db['electron_lifetimes'])

Success! we have inserted data into our own mongo database!